In [6]:
import os, sys, re, requests

import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import time
from tqdm.notebook import tqdm
from distributed import Client, LocalCluster
# import cgls_land as CGL
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gp
import numpy as np
import hvplot.xarray
import zarr
# %run ./functions.ipynb

plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 175

In [7]:
cluster = LocalCluster()
client = Client(cluster)

C:\Users\Pier\miniconda3\envs\FOSS4G\lib\site-packages\distributed\node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 62376 instead
  warnings.warn(


In [ ]:
Download some data from the Copernicus Global Land Service, to acheive it registration is needed selecting on the top right of the [legacy porta](https://land.copernicus.vgt.vito.be/PDF/portal/Application.html) 'Register' following the instructions.
Data can be downloaded as well directyl from [Copernicus Open Access Hub](https://scihub.copernicus.eu/) but data are not based on atmospherically and BRDF corrected data.

Once you have the credential a session can be instanziated

In [8]:
user = ''
psw = ''
CGL_session = CGL.Session(user, psw)

NameError: name 'CGL' is not defined

In [ ]:
To obtain the collections availabe a list_collections() can be used

In [9]:
products = CGL_session.list_collections()

NameError: name 'CGL_session' is not defined

In [16]:
products

['albh_v1_1km',
 'aldh_v1_1km',
 'ba300_V3_333m',
 'ba300_v1_333m',
 'dmp300_v1_333m',
 'dmp_v2_1km',
 'fapar300_v1_333m',
 'fapar_v1_1km',
 'fapar_v2_1km',
 'fcover300_v1_333m',
 'fcover_v1_1km',
 'fcover_v2_1km',
 'gdmp300_v1_333m',
 'gdmp_v2_1km',
 'lai300_v1_333m',
 'lai_v1_1km',
 'lai_v2_1km',
 'lst10-dc_v1_0.045degree',
 'lst10-dc_v2_0.045degree',
 'lst10-tci_v1_0.045degree',
 'lst10-tci_v2_0.045degree',
 'lst_v1_0.045degree',
 'lst_v2_0.045degree',
 'ndvi-lts_v2_1km',
 'ndvi300_v1_333m',
 'ndvi300_v2_333m',
 'ndvi_stats_all',
 'ndvi_v2_1km',
 'ndvi_v3_1km',
 'ssm_v1_1km',
 'swi-static-ci_v1_0.1degree',
 'swi-static-dgg_v3_12.5km',
 'swi-static-fm_v1_0.1degree',
 'swi-static-tc_v1_0.1degree',
 'swi-static-wf_v1_0.1degree',
 'swi10_v3_0.1degree',
 'swi_v1_1km',
 'swi_v3_0.1degree',
 'tocr_v1_1km',
 'wb300_v1_333m']

In [ ]:
Once you have decided the product you are interested to you have to load the specific collection through the load_collection method

In [17]:
ndvi_300_coll = CGL_session.load_collection('ndvi300_v2_333m')

In [ ]:
To dowload locally the latest 'observation' data just have to define the output folder. through date a specific date or a slice between two dates can be used. the method .download() return the sucessfully downloaded dataset paths in a list

In [18]:
ndvi_300_path = ndvi_300_coll.download(path=r'L:\FOSS4G\data\obs', date=slice(pd.to_datetime('2021-01-01'), pd
                                                                              .to_datetime('2021-12-31')))

Skipping c_gls_NDVI300_202101010000_GLOBE_OLCI_V2.0.1.nc as already available
Skipping c_gls_NDVI300_202101110000_GLOBE_OLCI_V2.0.1.nc as already available
Skipping c_gls_NDVI300_202102010000_GLOBE_OLCI_V2.0.1.nc as already available
Skipping c_gls_NDVI300_202101210000_GLOBE_OLCI_V2.0.1.nc as already available
Skipping c_gls_NDVI300_202102110000_GLOBE_OLCI_V2.0.1.nc as already available
Skipping c_gls_NDVI300_202102210000_GLOBE_OLCI_V2.0.1.nc as already available
Skipping c_gls_NDVI300_202103010000_GLOBE_OLCI_V2.0.1.nc as already available
Skipping c_gls_NDVI300_202103110000_GLOBE_OLCI_V2.0.1.nc as already available
Skipping c_gls_NDVI300_202103210000_GLOBE_OLCI_V2.0.1.nc as already available
Skipping c_gls_NDVI300_202104010000_GLOBE_OLCI_V2.0.1.nc as already available
Skipping c_gls_NDVI300_202104110000_GLOBE_OLCI_V2.0.1.nc as already available
Skipping c_gls_NDVI300_202104210000_GLOBE_OLCI_V2.0.1.nc as already available
Skipping c_gls_NDVI300_202105010000_GLOBE_OLCI_V2.0.1.nc as alre

NetCDF Dataset opening

In [ ]:
Let's start to deal with data, to open some data we need to rely on the open_dataset funtion. Even if isn't necessary the engine is specified to highlight the fact that we are dealing with a netCDF file format. Later on we will modify the engine to read a dataset that's in another format.

In [ ]:
ndvi_300 = xr.open_dataset('FOSS4G/data/obs/c_gls_NDVI300_202207010000_GLOBE_OLCI_V2.0.1.nc', engine='netcdf')

In [ ]:
Having a look to the data we can highlight the structure of it. 


In [ ]:
ndvi_300

In [ ]:
#ndvi_300.NDVI.plot()

how to compute the memory needed to load a single DataArray is really important to avoid issues like kill the kernell

In [ ]:
print(f'{ndvi_300.NDVI.size * 9.31e-10} GB')

Having a look to the total ammont of free memory a question is needed... how data can be loaded if the system doesn't have enougt memory ? 

In [ ]:
! free -m -h

define chunks (int or dict) ( options : 'px', '5MB', 'auto', '-1')

In [13]:
xr.open_mfdataset(r'L:\FOSS4G\data\obs\*.nc', chunks={'time':-1, 'lat': 1272, 'lon':3270})

,Array,Chunk
Bytes,784.28 GiB,15.87 MiB
Shape,"(37, 47040, 120960)","(1, 1272, 3270)"
Count,101343 Tasks,50653 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,784.28 GiB,15.87 MiB
Shape,"(37, 47040, 120960)","(1, 1272, 3270)"
Count,101343 Tasks,50653 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [ ]:
ndvi_300 = xr.open_dataset(ndvi_300_path[0], chunks={'time':-1, 'lat': 1272, 'lon':3270})

In [ ]:
ndvi_300

In [ ]:
Select a single Variable

In [ ]:
ndvi_300.NDVI

Selection methods
.sel 
.isel

selection of an Area Of Intereset throgh a bounding box

In [ ]:
ndvi_AOI = ndvi_300.NDVI.sel(lat=slice(44.5,42.3) ,lon=slice(9.65,12.45))

In [ ]:
# plot with matplot lib

In [ ]:
ndvi_AOI.plot(cmap="RdYlGn")

In [ ]:
ndvi_300.NDVI.sel(lat=42.50, lon=10., method='nearest').values

In [ ]:
ndvi_300.NDVI.attrs

values are a little bit odd in comparison with the max values from the PUM

## NDVI characteristics from the Product User Manual (PUM)

| layer name  | description                             | physical min | physical max | digital max | scaling | offset | No Data  |
|-------------|-----------------------------------------|--------------|--------------|-------------|---------|--------|----------|
| ndvi        | normalized difference vegetation index  | -0.08        | 0.92         | 250         | 1/250   | -0.08  | 254, 255 |
| ndvi_unc    | uncertainty on ndvi                     | 0            | 1            | 1000        | 1/1000  | 0      | 65535    |
| nobs        | number of observations                  | 0	           | 32           | 32          | 1       | 0      | 255      |
| qflag       | bitwise quality flag                    | -            | -            | 254         | 1       | 0      | 255      |


from : [Copernicus Global Land Service NDVI 300 V2.0.1](https://land.copernicus.eu/global/sites/cgls.vito.be/files/products/CGLOPS1_PUM_NDVI300m-V2_I1.20.pdf)

how to use the scale factor included within the dataset

In [ ]:
ndvi_U = xr.open_dataset(ndvi_300_path[0], mask_and_scale=False)

In [ ]:
ndvi_U_AOI = ndvi_U.NDVI.sel(lat=slice(44.5,42.3) ,lon=slice(9.65,12.45))

how to plot

In [ ]:
ndvi_U_AOI.plot(cmap="RdYlGn")

how to obtain values from the underlying numpy array

In [ ]:
ndvi_U_AOI.sel(lat=42.50, lon=10., method='nearest').data


mask values that are not in the valid range

In [ ]:
ndvi_8bit_Mask = ndvi_U_AOI.where(ndvi_U_AOI <= 250.)

In [ ]:
ndvi_8bit_Mask.sel(lat=42.50, lon=10., method='nearest').values

In [ ]:
To plot 

In [ ]:
ndvi_8bit_Mask.plot()

scale values and adding the offset

In [ ]:
ndvi_scaled = ndvi_8bit_Mask * (1/250) - 0.08

Check if min and max values are correct

In [ ]:
print(f'min: {ndvi_scaled.min().values}, \n'
      f'max: {ndvi_scaled.max().values}' )

make a comparison with the dataarray masked and scaled at the source

In [ ]:
ndvi_scaled.sel(lat=44., lon=11., method='nearest').values

In [ ]:
ndvi_300.NDVI.sel(lat=44., lon=11., method='nearest').values

In [ ]:
ndvi_mask = ndvi_AOI.where(ndvi_AOI<0.92, np.nan )

In [ ]:
ndvi_mask.plot(cmap="RdYlGn", vmin=-0.08, vmax=0.92)

Mask data according to a polygon

In [ ]:
GAUL = gp.read_file(r'L:\FOSS4G\data\GAUL\gaul1_asap.shp')

In [ ]:
GAUL

select the geometry of interest

In [ ]:
tuscany_poly = GAUL[GAUL.name1 == 'Toscana'].geometry

introduction of rioxarray

In [ ]:
ndvi_AOI = ndvi_AOI.rio.write_crs(4326)

In [ ]:
ndvi_AOI = ndvi_AOI.rio.clip(tuscany_poly, crs=4326)

In [ ]:
import matplotlib.colors as colors

divnorm = colors.TwoSlopeNorm(vmin=-0.8, vcenter=0.14, vmax=0.92)
ndvi_AOI.plot(cmap="RdYlGn", norm=divnorm )

Aggregate dataset on the time dimension

In [3]:
tuscany = xr.open_mfdataset(r'L:\FOSS4G\data\obs\tuscany\*.nc')

In [ ]:
tuscany.NDVI.ihvplot()

In [ ]:
tuscany.NDVI.isel(time=1).hvplot(cmap='fire')

In [4]:
tuscany.NDVI.hvplot(groupby='time')

:DynamicMap   [time]
   :Image   [lon,lat]   (NDVI)

Download NDVI long term statistics

In [ ]:
ndvi_lts_coll = CGL_session.load_collection('ndvi_stats_all')

In [ ]:
ndvi_lts_coll.infos

In [ ]:
ndvi_lts_files = ndvi_lts_coll.download(date=slice(pd.to_datetime('1999-01-01'), pd.to_datetime('1999-12-21')),
                                        path=r'L:\FOSS4G\data\LTS', sensor='VGT-PROBAV')

In [ ]:
xr.open_dataset(ndvi_lts_files[0])

In [ ]:
ndvi_lts = xr.open_mfdataset(ndvi_lts_files, combine='nested', concat_dim=['time'])

In [ ]:
ndvi_lts = ndvi_lts.rio.write_crs(4326)

In [ ]:
ndvi_lts = ndvi_lts.sel(lat=slice(44.5,42.3) ,lon=slice(9.65,12.45))

In [ ]:
ndvi_lts = ndvi_lts.rio.clip(tuscany_poly, crs=4326)

In [ ]:
ndvi_lts = ndvi_lts.chunk({'time':-1, 'lat':'50MB', 'lon':'50MB'})

In [ ]:
ndvi_lts.attrs

In [ ]:
ndvi_lts.to_zarr(r'L:\FOSS4G\data\LTS\c_gls_NDVI-LTS_1999-2019-Tuscany_VGT-PROBAV_V3.zarr', mode='w' )

Load a zarr file (introduction to zarr format)

In [ ]:
ndvi_lts = xr.open_dataset(r'L:\FOSS4G\data\LTS\c_gls_NDVI-LTS_1999-2019-Tuscany_VGT-PROBAV_V3.zarr', engine='zarr')

In [ ]:
ndvi_lts['min'].sel(lat=44., lon=11., method='nearest').plot()

In [ ]:
# reindex according to a 2022 date schema 

In [ ]:
Calculate a graph to visualize the difference for a single point between the LTS and the latest observations

In [ ]:
Create a stress map


In [ ]:
names = ndvi_300.QFLAG.sel(lat=slice(44.5,44.0) ,lon=slice(9.65,10.)).attrs['flag_meanings'].split(' ')
values = ndvi_300.QFLAG.sel(lat=slice(44.5,44.0) ,lon=slice(9.65,10.)).attrs['flag_masks']
dictionary = dict(zip(names ,values))
dictionary